In [1]:
from collections import deque, defaultdict, Counter
from heapq import heapify, heappush, heappop
import numpy as np
from copy import deepcopy
import math
import time
from functools import cache, reduce, cmp_to_key
import graphviz
from itertools import product
import matplotlib.pyplot as plt
from bisect import bisect_left, bisect_right
import json

In [2]:
with open("./data/day17.txt") as f:
    jets = f.readline().rstrip()
len(jets)

10091

In [3]:
class Block:
    def __init__(self, type, start_row, start_col) -> None:
        if type == 0: # horizontal line
            self.occupied = set([(start_row, start_col+i) for i in range(4)])
        elif type == 1: # plus
            self.occupied = set([(start_row+1+i, start_col+1+j) for i, j in [(0, 0), (-1, 0), (1, 0), (0, -1), (0, 1)]])
        elif type == 2: # plus
            self.occupied = set([(start_row+i, start_col+j) for i, j in [(0, 0), (0, 1), (0, 2), (1, 2), (2, 2)]])
        elif type == 3: # vertical line
            self.occupied = set([(start_row+i, start_col) for i in range(4)])
        elif type == 4: # square
            self.occupied = set([(start_row+i, start_col+j) for i, j in [(0, 0), (0, 1), (1, 0), (1, 1)]])
    
    def max_row(self):
        return max(map(lambda x: x[0], self.occupied))

    def jet(self, direction, blocked):
        new_occupied = set()
        if direction == '<':
            for row, col in self.occupied:
                if col-1 < 0:
                    return False
                if (row, col-1) in blocked:
                    return False
                new_occupied.add((row, col-1))
            self.occupied = new_occupied
        else:
            for row, col in self.occupied:
                if col+1 >= 7:
                    return False
                if (row, col+1) in blocked:
                    return False
                new_occupied.add((row, col+1))
            self.occupied = new_occupied
        return True
    
    def drop(self, blocked):
        new_occupied = set()
        for row, col in self.occupied:
            if row-1 <= 0:
                return False
            if (row-1, col) in blocked:
                return False
            new_occupied.add((row-1, col))
        self.occupied = new_occupied
        return True

In [4]:
blocked = set()

jet_ind = 0
max_row = 0
for i in range(2022):
    curr_block = Block(i%5, max_row+4, 2)
    flag = True
    while flag:
        _ = curr_block.jet(jets[jet_ind], blocked)
        jet_ind = (jet_ind+1)%len(jets)
        flag = curr_block.drop(blocked)
    for pos in curr_block.occupied:
        blocked.add(pos)
    max_row = max(max_row, curr_block.max_row())

In [5]:
for row in range(9, 0, -1):
    s = ''
    for col in range(7):
        s += '#' if (row, col) in blocked else '.'
    print(s)

...#...
.####..
.####..
.####..
.####..
.##....
.###...
..#....
..####.


In [6]:
max_row

3175

# Part 2

Since we have no way of simulating dropping 1000000000000 blocks, we clearly need to find some looping behavior.

We can somewhat rephrase the situation to make finding loops understandable. Instead of thinking about the height of the pile after dropping `n` blocks, let's consider the height of the settled pile after `n` jet gusts and blocks being moved down by one step. If there is looping in this simulation, there will be looping at some multiple of `len(jets)` intervals. Let's get some values of this function at `len(jets)` intervals and see if there are indications of looping going on.

In [7]:
blocked = set()

res = []

jet_ind = 0
max_row = 0
for i in range(10*2022):
    curr_block = Block(i%5, max_row+4, 2)
    flag = True
    while flag:
        if jet_ind == 0:
            res.append((i, max_row))
        _ = curr_block.jet(jets[jet_ind], blocked)
        jet_ind = (jet_ind+1)%len(jets)
        flag = curr_block.drop(blocked)
    for pos in curr_block.occupied:
        blocked.add(pos)
    max_row = max(max_row, curr_block.max_row())

In [8]:
print('Heights of the pile:', [x[1] for x in res])
print('Height increases:', [res[i+1][1]-res[i][1] for i in range(len(res)-1)])
print()
print('Dropped and settled rocks:', [x[0] for x in res])
print('Increases in number of rocks:', [res[i+1][0]-res[i][0] for i in range(len(res)-1)])

Heights of the pile: [0, 2750, 5487, 8224, 10961, 13698, 16435, 19172, 21909, 24646, 27383, 30120]
Height increases: [2750, 2737, 2737, 2737, 2737, 2737, 2737, 2737, 2737, 2737, 2737]

Dropped and settled rocks: [0, 1743, 3503, 5263, 7023, 8783, 10543, 12303, 14063, 15823, 17583, 19343]
Increases in number of rocks: [1743, 1760, 1760, 1760, 1760, 1760, 1760, 1760, 1760, 1760, 1760]


So, after 1743 rocks have been dropped, dropping 1760 more rocks seem to always add 2737 to the height of the pile.

In [9]:
target = 1000000000000

loop_start = 1743
loop_length = 1760
loop_increase = 2737

# target = loop_start + loops*loop_length + remainder
loops = (target-loop_start)//loop_length
remainder = (target-loop_start)%loop_length

loops, remainder

(568181817, 337)

In [10]:
blocked = set()
jet_ind = 0
max_row = 0
for i in range(loop_start+remainder):
    curr_block = Block(i%5, max_row+4, 2)
    flag = True
    while flag:
        _ = curr_block.jet(jets[jet_ind], blocked)
        jet_ind = (jet_ind+1)%len(jets)
        flag = curr_block.drop(blocked)
    for pos in curr_block.occupied:
        blocked.add(pos)
    max_row = max(max_row, curr_block.max_row())

init_rocks = max_row

print('Result =', init_rocks+loops*loop_increase)

Result = 1555113636385


Hacky, very hacky, but it's the right answer.